In [14]:
import os
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv('soci_train.csv')
df = df.drop(['Unnamed: 0'], axis = 1)

In [27]:
print("Dataset has {} entries and {} features".format(*df.shape))

Dataset has 1192 entries and 26 features


In [28]:
df

,country,isonum,isocode,year,rank_soci,soci,strange,strange_year,polic,polic_year,...,vtf,vtf_year,finhelp,finhelp_year,volunt,volunt_year,help,help_year,donate,donate_year
0,Afghanistan,4,AFG,2007,144,37.691658,***,2008,***,2008,...,22.216499,2007,***,2011,***,2008,***,2008,***,2008
1,Angola,24,AGO,2007,107,44.954544,***,2011,***,2011,...,0.000000,2007,***,2012,***,2011,***,2011,***,2011
2,Albania,8,ALB,2007,124,41.740612,***,2009,***,2009,...,46.293499,2007,***,2012,***,2009,***,2009,***,2009
3,United Arab Emirates,784,ARE,2007,51,52.181824,***,2009,***,--,...,0.000000,2007,***,2012,***,2009,***,2009,***,2009
4,Argentina,32,ARG,2007,78,48.400772,***,2007,***,2007,...,65.816994,2007,***,2012,***,2007,***,2007,***,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Vietnam,704,VNM,2014,73,49.223934,***,2014,***,2013,...,14.926501,2014,***,2014,***,2014,***,2014,***,2014
1188,Yemen,887,YEM,2014,149,34.863636,***,2014,***,2014,...,0.000000,2014,***,2014,***,2014,***,2014,***,2014
1189,South Africa,710,ZAF,2014,30,57.354820,***,2014,***,2014,...,69.806000,2014,***,2014,***,2014,***,2014,***,2014
1190,Zambia,894,ZMB,2014,66,50.936478,***,2014,***,2014,...,45.602501,2014,***,2014,***,2014,***,2014,***,2014


In [29]:
y = df['soci']
df = df.drop(['rank_soci', 'soci'], axis = 1)

In [30]:
from featuretools.selection import remove_low_information_features, remove_highly_null_features, remove_single_value_features, remove_highly_correlated_features

df = remove_low_information_features(df)

df = remove_highly_null_features(df)

df = remove_single_value_features(df)

df = remove_highly_correlated_features(df)

X = df

In [31]:
import evalml
import woodwork as ww

problem_type = 'regression'
objective =  'auto'

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


automl = evalml.automl.AutoMLSearch(X_train, y_train, problem_type=problem_type, objective = objective)
automl.search()

best_pipeline = automl.best_pipeline

Using default limit of max_batches=1.

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: extra_trees, xgboost, catboost, random_forest, linear_model, lightgbm, decision_tree



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mean Baseline Regression Pipeline        Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean R2: -0.000
High coefficient of variation (cv >= 0.2) within cross validation scores. Mean Baseline Regression Pipeline may not perform as estimated on unseen data.
Batch 1: (2/9) Decision Tree Regressor w/ Imputer + ... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean R2: 0.464
Batch 1: (3/9) Extra Trees Regressor w/ Imputer + On... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean R2: 0.455
Batch 1: (4/9) XGBoost Regressor w/ Imputer + One Ho... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean R2: 0.736
Batch 1: (5/9) CatBoost Regressor w/ Imputer            Elapsed:00:30
	Starting cross validation
	Finished cross validation - mean R2: 0.214
High coefficient of variation (cv >= 0.2) within cross validation scores. CatBoost Regressor w/ Imputer may not perform as estimated on unsee

In [32]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,XGBoost Regressor w/ Imputer + One Hot Encoder,7.361769e-01,7.592078e-01,6.901876e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,5,Random Forest Regressor w/ Imputer + One Hot E...,5.558791e-01,5.402510e-01,5.211777e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,6,LightGBM Regressor w/ Imputer + One Hot Encoder,5.117439e-01,5.141417e-01,4.798055e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,1,Decision Tree Regressor w/ Imputer + One Hot E...,4.635979e-01,4.625880e-01,4.346738e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,2,Extra Trees Regressor w/ Imputer + One Hot Enc...,4.549566e-01,4.759850e-01,4.265735e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,8,Elastic Net Regressor w/ Imputer + One Hot Enc...,2.797968e-01,2.863813e-01,2.623798e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,4,CatBoost Regressor w/ Imputer,2.136879e-01,1.483318e-01,2.004097e+05,True,{'Imputer': {'categorical_impute_strategy': 'm...
7,0,Mean Baseline Regression Pipeline,-1.066788e-04,-6.333924e-05,0.000000e+00,True,{'Baseline Regressor': {'strategy': 'mean'}}
8,7,Linear Regressor w/ Imputer + One Hot Encoder ...,-5.845153e+17,-1.753546e+18,-5.479210e+23,True,{'Imputer': {'categorical_impute_strategy': 'm...


In [33]:
conda install python-graphviz

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - ^C
failed with repodata from current_repodata.json, will retry with next repodata source.

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [34]:
best_pipeline.graph()

RuntimeError: To graph entity sets, a graphviz backend is required.
Install the backend using one of the following commands:
  Mac OS: brew install graphviz
  Linux (Ubuntu): sudo apt-get install graphviz
  Windows: conda install python-graphviz


In [35]:
best_pipeline.feature_importance

,feature,importance
0,vtf,0.092835
1,finhelp_year_2014,0.078310
2,help_year_2007,0.066701
3,country_Peru,0.038785
4,polic_year_2014,0.036928
...,...,...
59,voice_year_2011,0.000000
60,help_year_--,0.000000
61,help_year_2009,0.000000
62,help_year_2013,0.000000


In [36]:
predictions = best_pipeline.predict(X_test)
predictions = predictions.to_series()

In [37]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('R2: ' + str(r2_score(y_test,  predictions)))
print('MSE: ' + str(mean_squared_error(y_test, predictions, squared = True)))
print('RMSE: ' + str(mean_squared_error(y_test, predictions, squared = False)))
print('MAE: ' + str(mean_absolute_error(y_test, predictions)))

R2: 0.7924524995839047
MSE: 12.27921246305344
RMSE: 3.504170724016374
MAE: 2.671331693662093
